## Guardrails
Guardrails w LangChain to mechanizmy pilnujące, aby odpowiedzi modelu spełniały określone wymagania techniczne lub formatowe.
Na przykład JsonValidityEvaluator sprawdza, czy zwrócony tekst jest poprawnym JSON-em – jeśli struktura jest niepoprawna, evaluator zwróci błąd.
Dzięki temu można automatycznie wykrywać i odrzucać odpowiedzi, które nie nadają się do dalszego przetwarzania.

### JSON Format Validator
Sprawdza, czy wygenerowana odpowiedź jest poprawnym JSON-em

In [5]:
from langchain.evaluation import JsonValidityEvaluator

evaluator = JsonValidityEvaluator()
print(evaluator.evaluate_strings(prediction='{"x": 1}'))      # poprawne
print(evaluator.evaluate_strings(prediction='{x: 1}'))        # błąd


{'score': 1}
{'score': 0, 'reasoning': 'Expecting property name enclosed in double quotes: line 1 column 2 (char 1)'}


### JsonEqualityEvaluator
Sprawdza równość JSON-ów po sparsowaniu (kolejność kluczy w JSON nie ma znaczenia)

In [6]:
from langchain.evaluation import JsonEqualityEvaluator

evaluator = JsonEqualityEvaluator()
print(evaluator.evaluate_strings(
    prediction='{"a":1,"b":[2,3]}',
    reference='{"b":[2,3],"a":1}',
))


{'score': True}


### RegexMatchEvaluator
Sprawdza dopasowanie do wyrażenia regularnego

In [7]:
from langchain.evaluation import RegexMatchStringEvaluator

evaluator = RegexMatchStringEvaluator()
print(evaluator.evaluate_strings(
    prediction="Order ID: ABC-1234",
    reference=r"^Order ID: [A-Z]{3}-\d{4}$",
))

{'score': 1}


### Fallback Messages Validator
Kiedy główny model zawiedzie (rate limit / błąd) automatycznie użyj modelu zapasowego.

In [12]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

primary = ChatOpenAI(model="gpt-4o-mini", max_retries=0)  # wyłącz retry, by fallback zadziałał
backup  = ChatOpenAI(model="gpt-3.5-turbo")

chain = primary.with_fallbacks([backup])

print(chain.invoke("Opisz w 1 zdaniu Python."))


content='Python to wszechstronny, wysokopoziomowy język programowania, który charakteryzuje się czytelnością składni, dużą biblioteką standardową i szerokim zastosowaniem w różnych dziedzinach, takich jak web development, analiza danych, sztuczna inteligencja i automatyzacja.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 16, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CJE9iPMlz9yPrf3dhfEkV1mIb2Z5y', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--19bf2938-84e2-49cf-b8f1-0c83ab5388eb-0' usage_metadata={'input_tokens': 16, 'output_tokens': 70, 'total_tokens': 86, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'o

### Token Limit
Śledzenie i przycinanie historii do limitu tokenów, żeby nie przekroczyć kontekstu modelu.

In [16]:
import json
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.messages.utils import trim_messages, count_tokens_approximately
from langchain_openai import ChatOpenAI

messages = [
    SystemMessage(content="Jesteś pomocnym asystentem."),
    HumanMessage(content="(tu długa historia rozmowy / wiele wiadomości...)"),
]

trimmed = trim_messages(
    messages,
    strategy="last",
    token_counter=count_tokens_approximately,
    max_tokens=256,
    start_on="human",
    include_system=True,
)

llm = ChatOpenAI(model="gpt-4o-mini")
print(json.dumps(llm.invoke(trimmed).response_metadata, indent=4))


{
    "token_usage": {
        "completion_tokens": 60,
        "prompt_tokens": 33,
        "total_tokens": 93,
        "completion_tokens_details": {
            "accepted_prediction_tokens": 0,
            "audio_tokens": 0,
            "reasoning_tokens": 0,
            "rejected_prediction_tokens": 0
        },
        "prompt_tokens_details": {
            "audio_tokens": 0,
            "cached_tokens": 0
        }
    },
    "model_name": "gpt-4o-mini-2024-07-18",
    "system_fingerprint": "fp_560af6e559",
    "id": "chatcmpl-CJEB7VOt6o9l0iCXrU4wQW8V7GhTm",
    "service_tier": "default",
    "finish_reason": "stop",
    "logprobs": null
}


### Word Limit
poproś o max N słów, po generacji policz słowa. W razie przekroczenia – skróć odpowiedź.

In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

limit = 25
prompt = f"Napisz streszczenie w MAKS {limit} słowach: Czym jest uczenie maszynowe?"

resp = llm.invoke(prompt).content
if len(resp.split()) > limit:
    # szybka korekta – poproś model o skrócenie do limitu
    resp = llm.invoke(f"Skróć to do maks {limit} słów, bez dopisków:\n\n{resp}").content

print(resp)


Uczenie maszynowe to dziedzina sztucznej inteligencji, która umożliwia komputerom uczenie się z danych i doskonalenie swoich działań bez programowania.
